<a href="https://colab.research.google.com/github/SilverFoxMedia-Tesis/IA_02/blob/main/Modelo_Red_Neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Lectura de datos
train = pd.read_csv('/ObesityDataSet_raw_and_data_sinthetic.csv')
test = pd.read_csv('/ObesityDataSet_raw_and_data_sinthetic.csv')

In [ ]:
peso = 80  # kg
altura = 175  # cm
edad = 22  # años
genero = 'hombre'  # 'hombre' o 'mujer'
nivel_actividad = 'ligero'  # 'ninguno', 'ligero', 'moderado', 'fuerte', 'muy fuerte'
NObeyesdad = 'Insufficient_Weight' #'Insufficient_Weight' , 'Normal_Weight' , 'Overweight_Level_I' , 'Overweight_Level_II' , 'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III',


In [ ]:
def calcular_TMB_MIFFLIN(Weight, Height, Age, Gender, FAF):

  # Cálculo de TMB según el género
  if Gender.lower() == 'Male':
      partial_Tmb = (10 * Weight) + (6.25 * Height) - (5 * Age) + 5
  else: # Female
      partial_Tmb = (10 * Weight) + (6.25 * Height) - (5 * Age) - 161

  # Multiplicador según el nivel de actividad
  multiplicador = {
      0: 1.2,
      1: 1.375,
      2: 1.725,
      3: 1.9
  }

  # Calorías necesarias según el nivel de actividad
  TMB = partial_Tmb * multiplicador[FAF]

  return TMB

In [ ]:
calcular_TMB_MIFFLIN(peso, altura, edad, genero, nivel_actividad)

2146.5

In [ ]:
def calcular_TMB_FAO_ONU(Weight, Age, Gender):

    if Gender == 'Male':
        if Age >= 0 and Age <= 3:
            TMB = (60.9 * Weight) - 54
        elif Age >= 4 and Age <= 10:
            TMB = (22.7 * Weight) + 495
        elif Age >= 11 and Age <= 18:
            TMB = (17.5 * Weight) + 651
        elif Age >= 19 and Age <= 30:
            TMB = (15.3 * Weight) + 679
        elif Age >= 31 and Age <= 60:
            TMB = (11.6 * Weight) + 879
        else: # Age > 60
            TMB = (13.5 * Weight) + 487
    else: # Gender == 'mujer'
        if Age >= 0 and Age <= 3:
            TMB = (61.0 * Weight) - 51
        elif Age >= 4 and Age <= 10:
            TMB = (22.5 * Weight) + 499
        elif Age >= 11 and Age <= 18:
            TMB = (12.2 * Weight) + 746
        elif Age >= 19 and Age <= 30:
            TMB = (14.7 * Weight) + 496
        elif Age >= 31 and Age <= 60:
            TMB = (8.7 * Weight) + 829
        else: # edad > 60
            TMB = (10.5 * Weight) + 596

    return TMB

In [ ]:
calcular_TMB_FAO_ONU(peso, edad, genero)

1903.0

In [ ]:
def calcular_TMB_harris_benedict_simplificada(Weight, Height, Age, Gender, FAF):
    if Gender == 'Male':
        partial_TMB = 66.5 + (13.75 * Weight) + (5.003 * Height) - (6.78 * Age)
    else:
        partial_TMB = 655 + (9.56 * Weight) + (1.85 * Height) - (4.68 * Age)
  # Multiplicador según el nivel de actividad
    multiplicador = {
      0: 1.2,
      1: 1.375,
      2: 1.725,
      3: 1.9
    }

    # Calorías necesarias según el nivel de actividad
    TMB = partial_TMB * multiplicador[FAF]

    return TMB

In [ ]:
calcular_TMB_harris_benedict_simplificada(peso, altura, edad, genero,nivel_actividad)



2602.689375

In [ ]:
def calcular_TMB(Weight, Height, Age, Gender, FAF, NObeyesdad):
    if NObeyesdad in ['Insufficient_Weight', 'Normal_Weight']:
        return -1
    elif NObeyesdad in ['Obesity_Type_II', 'Obesity_Type_III']:
        return calcular_TMB_MIFFLIN(Weight, Height, Age, Gender, FAF)
    else:
        if FAF == 0:
            return calcular_TMB_FAO_ONU(Weight, Age, Gender)
        else:
            return calcular_TMB_harris_benedict_simplificada(Weight, Height, Age, Gender, FAF)

In [ ]:
calcular_TMB(Weight, Height, Age, Gender, FAF, NObeyesdad)

-1

In [ ]:
def calcular_macronutrientes(TMB):
    if TMB == -1:
      return "NO DISPONIBLE"
    else:
      c_carbohidratos = TMB * 0.50
      c_proteinas = TMB * 0.20
      c_grasas = TMB * 0.30

      # Conversión de calorías a gramos
      g_carbohidratos = c_carbohidratos / 4
      g_proteinas = c_proteinas / 4
      g_grasas = c_grasas / 9

      return {
          "Carbohidratos (g)": round(g_carbohidratos, 2),
          "Proteínas (g)": round(g_proteinas, 2),
          "Grasas (g)": round(g_grasas, 2)
      }


In [ ]:
calcular_macronutrientes(calcular_TMB(Weight, Height, Age, Gender, FAF, NObeyesdad))

'NO DISPONIBLE'

In [67]:
# Asegúrate de que 'FAF' es un entero y de que manejas los valores nulos o faltantes
train['FAF'] = train['FAF'].fillna(0).astype(int)

# Luego usa la función apply con la función lambda corregida
train['TMB'] = train.apply(lambda row: calcular_TMB(
    row['Weight'],
    row['Height'],
    row['Age'],
    row['Gender'],
    row['FAF'],
    row['NObeyesdad']
    ), axis=1)

def get_macronutrientes(TMB):
    return calcular_macronutrientes(TMB) if TMB != -1 else {"Carbohidratos (g)": None, "Proteínas (g)": None, "Grasas (g)": None}

# Suponiendo que la columna TMB ya ha sido creada correctamente
macronutrientes = train['TMB'].apply(get_macronutrientes)

# Convertir el diccionario de resultados en un DataFrame y unirlo al DataFrame original
macronutrientes_df = pd.DataFrame(macronutrientes.tolist(), index=train.index)
train = train.join(macronutrientes_df)



In [71]:
train

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad,TMB,Carbohidratos (g),Proteínas (g),Grasas (g)
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,...,no,0,1.000000,no,Public_Transportation,Normal_Weight,-1.000000,NaN,NaN,NaN
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,...,yes,3,0.000000,Sometimes,Public_Transportation,Normal_Weight,-1.000000,NaN,NaN,NaN
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,...,no,2,1.000000,Frequently,Public_Transportation,Normal_Weight,-1.000000,NaN,NaN,NaN
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,...,no,2,0.000000,Frequently,Walking,Overweight_Level_I,1877.999565,234.75,93.90,62.60
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,...,no,0,0.000000,Sometimes,Public_Transportation,Overweight_Level_II,2052.940000,256.62,102.65,68.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,...,no,1,0.906247,Sometimes,Public_Transportation,Obesity_Type_III,1455.978057,182.00,72.80,48.53
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,...,no,1,0.599270,Sometimes,Public_Transportation,Obesity_Type_III,1481.484634,185.19,74.07,49.38
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,...,no,1,0.646288,Sometimes,Public_Transportation,Obesity_Type_III,1477.058863,184.63,73.85,49.24
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,...,no,1,0.586035,Sometimes,Public_Transportation,Obesity_Type_III,1459.601402,182.45,72.98,48.65
